<a href="https://colab.research.google.com/github/emnordi/ce888/blob/master/Project2/Project2_Mushroom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import io
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
!git clone https://github.com/emnordi/ce888.git
! git pull
from ce888.Project2.utils import *
os.environ['KAGGLE_USERNAME'] = "emilnordin" # username from the json file

os.environ['KAGGLE_KEY'] = "fd1ae7405c909d36a1ecffea5cd2749a" # key from the json file

fatal: destination path 'ce888' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git


DOWNLOADING AND IMPORTING DATASET

In [2]:
!kaggle datasets download -d uciml/mushroom-classification

mushroom-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
data_mush = pd.read_csv('mushroom-classification.zip', compression='zip', header=0, sep=',', quotechar='"', encoding = "utf-8")

In [0]:
#Downsample to get 65% imbalance
mush_p_ind = data_mush[data_mush['class'] == 'p'].index
to_del = mush_p_ind[0:int(len(mush_p_ind)*0.43)]
data_mush_imbalanced = data_mush.drop(index=to_del)

#Convert string data to numerical
data_mush_imbalanced_ = pd.get_dummies(data_mush_imbalanced)
data_mush_imbalanced_ = data_mush_imbalanced_.drop('class_e', axis=1)

PRINTING DATASET

In [5]:
print(data_mush_imbalanced_)

      class_p  cap-shape_b  cap-shape_c  ...  habitat_p  habitat_u  habitat_w
1           0            0            0  ...          0          0          0
2           0            1            0  ...          0          0          0
4           0            0            0  ...          0          0          0
5           0            0            0  ...          0          0          0
6           0            1            0  ...          0          0          0
...       ...          ...          ...  ...        ...        ...        ...
8119        0            0            0  ...          0          0          0
8120        0            0            0  ...          0          0          0
8121        0            0            0  ...          0          0          0
8122        1            0            0  ...          0          0          0
8123        0            0            0  ...          0          0          0

[6441 rows x 116 columns]


DISPLAY AMOUNT OF ROWS EACH CLASS HAS, AND IMBALANCE OF DATASET

In [6]:
print(data_mush_imbalanced_['class_p'].value_counts())
print("Imbalance:", data_mush_imbalanced_['class_p'].value_counts()[0]/sum(data_mush_imbalanced_.class_p.value_counts()))

0    4208
1    2233
Name: class_p, dtype: int64
Imbalance: 0.6533147026859183


BASE RESULTS

In [0]:
#Create decision tree and random forest classifiers
tree_clf = tree.DecisionTreeClassifier(random_state=0)
rf_clf = RandomForestClassifier(max_depth=2, random_state=0)

In [8]:
#Mushroom dataset base scores DT
feat_mush = np.array(data_mush_imbalanced_.drop('class_p', axis=1))
lab_mush = np.array(data_mush_imbalanced_['class_p'])

base = Utils.evaluatePerformance(tree_clf, feat_mush, lab_mush)

AttributeError: ignored

In [0]:
#Mushroom dataset base scores RF
Utils.evaluatePerformance(rf_clf, feat_mush, lab_mush);

DOWNSAMPLING THE DATASETS AND RETRIEVING NEW PERFORMANCE RESULTS

In [0]:
#Downsample MUSHROOM DATASET to get 50%
mush_p_ind_ = data_mush_imbalanced_[data_mush_imbalanced_['class_p'] == 0].index
to_del = mush_p_ind_[0:int(len(mush_p_ind_)*0.46)]
data_mush_imbalanced_test = data_mush_imbalanced_.drop(index=to_del)

print(data_mush_imbalanced_test['class_p'].value_counts())
print("Imbalance:", data_mush_imbalanced_test['class_p'].value_counts()[0]/(data_mush_imbalanced_test['class_p'].value_counts()[1]+(data_mush_imbalanced_test['class_p'].value_counts()[0])))

In [0]:
#MUSHROOM DATASET downsampling scores DT
feat_mush_test = data_mush_imbalanced_test.drop('class_p', axis=1)
lab_mush_test = data_mush_imbalanced_test['class_p']

downsampled = evaluatePerformance(tree_clf, feat_mush_test, lab_mush_test)

In [0]:
#MUSHROOM DATASET downsampling scores RF
Utils.evaluatePerformance(rf_clf, feat_mush_test, lab_mush_test);

OVERSAMPLING AND TEST RESULTS

In [0]:
#OVERSAMPLING MUSHROOM DATASET
max_ = data_mush_imbalanced_['class_p'].value_counts().max()

temp = [data_mush_imbalanced_]
for class_index, group in data_mush_imbalanced_.groupby('class_p'):
    temp.append(group.sample(max_-len(group), replace=True))
data_mush_over = pd.concat(temp)

print(data_mush_over['class_p'].value_counts())
print("Imbalance:", data_mush_over['class_p'].value_counts()[0]/(data_mush_over['class_p'].value_counts()[1]+(data_mush_over['class_p'].value_counts()[0])))

In [0]:
#MUSHROOM DATASET oversampling scores DT
feat_mush_over = data_mush_over.drop('class_p', axis=1)
lab_mush_over = data_mush_over['class_p']

oversampled = evaluatePerformance(tree_clf, feat_mush_over, lab_mush_over)

In [0]:
#MUSHROOM DATASET oversampling scores RF
Utils.evaluatePerformance(rf_clf, feat_mush_over, lab_mush_over);

USING NEW ALGORITHM TO CLASSIFY THE DATA

In [0]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.cluster import adjusted_rand_score

In [0]:
#MUSHROOM DATASET
skf = StratifiedKFold(n_splits=10)
print(lab_mush.shape)

Silhouette method & Elbow method - amount of cluster determination

In [0]:
#Initialise random forest classifier and lists to hold performance for each permutation
random_forest_clf = RandomForestClassifier(max_depth=2, random_state=0)
rf_classifiers = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

In [0]:
for train, test in skf.split(feat_mush, lab_mush):
  test_labels = []

  #Retrieve the predicted amount of clusters and the best performing KMeans model
  n_clusters, final_model = Utils.cluster_and_model(feat_mush[train], lab_mush[train])

  #Retrieve the clusters containing missclassified data and the indices of these clusters in the training data
  missclassified_cluster_data_index, missclassified_clusters = Utils.find_missclassified_clusters(n_clusters, lab_mush[train], final_model.labels_)
  rf_classifiers = [0]*n_clusters
  #Go through the clusters with missclassified data and train a random forest classifier for each
  for i in missclassified_clusters:
    f = feat_mush[train]
    f1 = f[missclassified_cluster_data_index[i], :]
    l  = lab_mush[train]
    l1 = np.take(l, missclassified_cluster_data_index[i])
    rf_classifiers[i] = random_forest_clf.fit(f1, l1)
  
  #For each point in test dataset, give it a cluster
  for x in feat_mush[test]:
    prediction = final_model.predict(np.array([x]))
    if prediction in missclassified_clusters:
      test_labels.append(rf_classifiers[prediction[0]].predict(np.array([x])))
    else:
      test_labels.append(prediction)
  accuracy_scores.append(accuracy_score(test_labels, lab_mush[test]))
  precision_scores.append(precision_score(test_labels, lab_mush[test]))
  recall_scores.append(recall_score(test_labels, lab_mush[test]))
  f1_scores.append(f1_score(test_labels, lab_mush[test]))
  

EVALUATING RESULTS FROM PROPOSED ALGORITHM

In [0]:
print("Accuracy:", np.mean(accuracy_scores), "+/-", np.std(accuracy_scores))
print("Precision:", np.mean(precision_scores), "+/-", np.std(precision_scores))
print("Recall:", np.mean(recall_scores), "+/-", np.std(recall_scores))
print("F1-Score:", np.mean(f1_scores), "+/-", np.std(f1_scores))

APPLYING SMOTE SAMPLING

In [0]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)

In [0]:
smote_feat, smote_lab = sm.fit_resample(feat_mush, lab_mush)
print("Imbalance:", np.bincount(smote_lab)[0]/(np.bincount(smote_lab)[1]+( np.bincount(smote_lab)[0])))

In [0]:
np.bincount(smote_lab)

EVALUATING RESULTS FROM SMOTE

In [0]:
#Decision tree results
smote = Utils.evaluatePerformance(tree_clf, smote_feat, smote_lab)

In [0]:
#Random forest results
Utils.evaluatePerformance(rf_clf, smote_feat, smote_lab);

In [0]:
Utils.boxplots("Mushroom dataset", base, oversampled, downsampled, f1_scores, smote)